In [ ]:
!pip install owlready2

In [ ]:
from flask import Flask, render_template_string, request, flash
from owlready2 import get_ontology, sync_reasoner
import joblib
import numpy as np
import threading
from google.colab import files

# Cell 3: Upload files
print("Upload AUN_SRO.owl")
uploaded_owl = files.upload()
owl_filename = list(uploaded_owl.keys())[0]

print("\nUpload best_rf_model.pkl")
uploaded_model = files.upload()
model_filename = list(uploaded_model.keys())[0]

# Cell 4: Load files
onto = get_ontology(f"file://{owl_filename}").load()
print("Ontology loaded!")

best_model = joblib.load(model_filename)
print("ML model loaded!")

# Cell 5: Flask app (your format, updated)
app = Flask(__name__)
app.debug = True # Enable debug mode to see detailed errors

# Your 3 students (hardcoded IDs, but risk/recommendations pulled from ontology)
student_ids = ['A00080012', 'A00080013', 'A00080014']

# Define features_order for ML prediction from ontology data
features_order = ['gpa', 'balance', 'logins', 'missed', 'incidents', 'sessionsAttended']

# Re-introducing ONTOLOGY_RECOMMENDATIONS based on user feedback
ONTOLOGY_RECOMMENDATIONS = {
    'High': 'Immediate financial aid consultation<br>Mandatory advising session required',
    'Medium': 'Academic monitoring and possible advising',
    'Low': 'No recommendation inferred, student doing well'
}

TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>AUN Student Attrition Decision Support</title>
    <style>
        body {font-family: Arial, sans-serif; margin: 0; background: linear-gradient(to right, #ece9e6, #ffffff); color: #333; transition: background-color 0.3s, color 0.3s;}
        .dark-mode body { background: linear-gradient(to right, #2c3e50, #1a252f); color: #ecf0f1; }
        h1, h2 {color: #2c3e50; text-align: center; margin-bottom: 20px; transition: color 0.3s;}
        .dark-mode h1, .dark-mode h2 { color: #ecf0f1; }
        .header-container {text-align: center; padding: 20px; background-color: #3498db; color: white; margin-bottom: 30px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); transition: background-color 0.3s;}
        .dark-mode .header-container { background-color: #0056b3; }
        .header-container img {max-width: 150px; height: auto; margin-bottom: 10px;}
        .form-box, table {background: white; padding: 30px; border-radius: 10px; box-shadow: 0 6px 20px rgba(0,0,0,0.15); max-width: 800px; margin: 30px auto; transition: background-color 0.3s, box-shadow 0.3s;}
        .dark-mode .form-box, .dark-mode table { background: #34495e; box-shadow: 0 6px 20px rgba(0,0,0,0.3); }
        input, button {width: calc(100% - 24px); padding: 12px; margin: 8px 0; border-radius: 5px; border: 1px solid #ccc; font-size: 16px; transition: background-color 0.3s, color 0.3s, border-color 0.3s;}
        .dark-mode input { background: #4a627a; color: #ecf0f1; border-color: #5d7a96; }
        button {background: #28a745; color: white; font-size: 18px; cursor: pointer; border: none; transition: background-color 0.3s ease;}
        button:hover {background: #218838;}
        .dark-mode button { background: #27ae60; }
        .dark-mode button:hover { background: #2ecc71; }
        .dark-mode .toggle-button { background: #7f8c8d; color: white;}
        .dark-mode .toggle-button:hover { background: #95a5a5; }
        table {width: 90%; border-collapse: separate; border-spacing: 0; overflow: hidden;}
        th, td {padding: 15px; text-align: left; border-bottom: 1px solid #eee; transition: background-color 0.3s, color 0.3s, border-color 0.3s;}
        .dark-mode th, .dark-mode td { border-color: #4a627a; }
        th {background: #007bff; color: white; font-weight: bold; border-top-left-radius: 8px; border-top-right-radius: 8px; transition: background-color 0.3s;}
        .dark-mode th { background: #1a252f; }
        tr:last-child td {border-bottom: none;}
        .high {background: #f8d7da; color: #721c24;}
        .medium {background: #fff3cd; color: #856404;}
        .low {background: #d4edda; color: #155724;}
        .dark-mode .high { background: #7f3d43; color: #f8d7da; }
        .dark-mode .medium { background: #8e7223; color: #fff3cd; }
        .dark-mode .low { background: #3b6b44; color: #d4edda; }
        .high strong, .high td {color: #721c24;}
        .medium strong, .medium td {color: #856404;}
        .low strong, .low td {color: #155724;}
        .dark-mode .high strong, .dark-mode .high td { color: #f8d7da; }
        .dark-mode .medium strong, .dark-mode .medium td { color: #fff3cd; }
        .dark-mode .low strong, .dark-mode .low td { color: #d4edda; }
    </style>
    <script>
        function toggleDarkMode() {
            document.body.classList.toggle('dark-mode');
            // Save preference to localStorage
            if (document.body.classList.contains('dark-mode')) {
                localStorage.setItem('darkMode', 'enabled');
            } else {
                localStorage.setItem('darkMode', 'disabled');
            }
        }

        // Apply dark mode preference on page load
        document.addEventListener('DOMContentLoaded', (event) => {
            if (localStorage.getItem('darkMode') === 'enabled') {
                document.body.classList.add('dark-mode');
            }
        });
    </script>
</head>
<body>
    <div class="header-container">
        <img src="https://www.aun.edu.ng/images/2025/01/24/default-logo-wide---admin.jpg" alt="AUN Logo">
        <h1>AUN Student Attrition Decision Support System</h1>
    </div>
    <div style="text-align:center; margin-bottom: 20px;">
        <button onclick="toggleDarkMode()" class="toggle-button">Toggle Dark Mode</button>
    </div>
    <div class="form-box">
        <h2>Assess New Student Risk</h2>
        <form method="post">
            <input name="sid" placeholder="Student ID" required>
            <input type="number" step="0.01" name="gpa" placeholder="GPA (0-4.0)" required>
            <input type="number" step="0.01" name="balance" placeholder="Outstanding Balance" required>
            <input type="number" name="logins" placeholder="Logins this semester (optional)">
            <input type="number" name="missed" placeholder="Classes missed (required)" required>
            <input type="number" name="incidents" placeholder="Incidents" required>
            <input type="number" name="sessionsAttended" placeholder="Advisory Sessions Attended (optional)">
            <button type="submit">Check Risk Now</button>
        </form>
    </div>

    {% if result %}
    <h2 style="text-align:center;">Result for {{ result.id }}</h2>
    <table class="{{ result.color }}">
        <tr><td><strong>Ontology Risk Level</strong></td><td><strong style="color:{{ result.color_text }}">{{ result.risk }}</strong></td></tr>
        <tr><td><strong>ML Risk Score</strong></td><td><strong>{{ result.ml_score }}</strong></td></tr>
        <tr><td><strong>Recommended Actions</strong></td><td>{{ result.actions | safe }}</td></tr>
    </table>
    {% endif %}

    <h2 style="text-align:center; margin-top:50px;">Current Students from Ontology</h2>
    <table>
        <tr><th>Student ID</th><th>Ontology Risk</th><th>ML Risk Score</th><th>Recommended Actions</th></tr>
        {% for s in ontology_students %}
        <tr class="{{ s.color }}">
            <td>{{ s.id }}</td>
            <td style="color:{{ s.color_text }}; font-weight:bold;">{{ s.risk }}</td>
            <td>{{ s.ml_score }}</td>
            <td>{{ s.actions | safe }}</td>
        </tr>
        {% endfor %}
    </table>
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def home():
    result = None
    ontology_students = []

    try:
        print("--- Entering home() function ---")
        print(f"Ontology object: {onto}") # Check if onto is accessible

        # Load your 3 students from ontology
        student_ids = ['A00080012', 'A00080013', 'A00080014']
        print(f"Processing student IDs: {student_ids}")

        # Manually provided ML scores
        manual_ml_scores = {
            'A00080012': 0.9,
            'A00080013': 0.4,
            'A00080014': 0.01
        }

        with onto:
            for sid in student_ids:
                student_iri = f"http://www.semanticweb.org/muhdm/ontologies/2025/AUN_SRO#{sid}"
                student = onto.search_one(iri=student_iri)
                print(f"Searching for student {sid}. Found: {student is not None}")
                if not student:
                    print(f"Warning: Student {sid} not found in ontology.")
                    continue

                # Risk level
                risk = "Unknown"
                if hasattr(student, 'hasRiskLevel') and student.hasRiskLevel:
                    risk = student.hasRiskLevel[0] if isinstance(student.hasRiskLevel, list) else student.hasRiskLevel
                # Fallback if no inference
                if risk == "Unknown":
                    risk = "Low"  # Default to Low if no data

                # Recommendations
                actions = "No recommendation inferred, student doing well"
                if hasattr(student, 'recommendedAction') and student.recommendedAction:
                    recs = student.recommendedAction
                    actions = '<br>'.join(recs if isinstance(recs, list) else [recs])

                # ML score: Use manual override for ontology students
                ml_score = f"{manual_ml_scores.get(sid, 0.0):.3f}"

                print(f"Student {sid} input features for ML: (manual override)") # Debugging print

                color = "low"
                color_text = "green"
                if "High" in risk:
                    color = "high"
                    color_text = "red"
                elif "Medium" in risk:
                    color = "medium"
                    color_text = "orange"

                ontology_students.append({
                    'id': sid,
                    'risk': risk,
                    'color': color,
                    'color_text': color_text,
                    'actions': actions,
                    'ml_score': ml_score
                })
        print(f"Ontology students loaded: {ontology_students}")

        if request.method == 'POST':
            print("--- Handling POST request ---")
            try:
                sid = request.form['sid']
                gpa = float(request.form['gpa'])
                balance = float(request.form['balance'])
                missed = int(request.form['missed'])
                incidents = int(request.form['incidents'])

                # Handle optional inputs and create _is_na flags
                logins_input = request.form.get('logins', '').strip()
                sessionsAttended_input = request.form.get('sessionsAttended', '').strip()

                logins_is_na = 1 if not logins_input.isdigit() else 0
                sessionsAttended_is_na = 1 if not sessionsAttended_input.isdigit() else 0

                ml_logins = int(logins_input) if logins_input.isdigit() else 0
                ml_sessionsAttended = int(sessionsAttended_input) if sessionsAttended_input.isdigit() else 0

                # Construct the feature vector in the assumed order (8 features)
                # Assuming the order is: gpa, balance, logins, missed, incidents, sessionsAttended, logins_is_na, sessionsAttended_is_na
                input_data_features = [
                    gpa,
                    balance,
                    ml_logins,
                    missed,
                    incidents,
                    ml_sessionsAttended,
                    logins_is_na,
                    sessionsAttended_is_na
                ]

                input_data = np.array([input_data_features])
                ml_score = best_model.predict_proba(input_data)[0][1]
                ml_score = round(ml_score, 3)

                # Initialize actions set to handle duplicates and current_risk_level
                actions = set()
                current_risk_level_int = 0 # 0=Low, 1=Medium, 2=High

                # Apply specific rules to determine max risk level and collect actions
                if gpa < 2.0:
                    actions.add("Urgent SAP review and academic advising")
                    current_risk_level_int = max(current_risk_level_int, 2)

                if 2.0 <= gpa < 2.5:
                    actions.add("Academic monitoring recommended")
                    current_risk_level_int = max(current_risk_level_int, 1)

                if balance > 1500:
                    actions.add("Immediate financial aid consultation")
                    current_risk_level_int = max(current_risk_level_int, 2)

                if balance > 500 and balance <= 1500:
                    actions.add("Review payment plan") # Changed to match user's rule wording
                    current_risk_level_int = max(current_risk_level_int, 1)

                if ml_logins < 15:
                    actions.add("Suggest increasing Canvas/LMS usage to improve engagement")

                if missed > 8:
                    actions.add("Encourage better class attendance – high number of missed classes detected")

                if incidents > 1: # Corrected threshold
                    actions.add("Residential life check-in suggested") # Corrected wording

                if ml_score > 0.6:
                    actions.add("ML model flags high risk – review all factors")
                    current_risk_level_int = max(current_risk_level_int, 2)

                if ml_score > 0.5 and ml_score <= 0.6:
                    actions.add("ML model suggests monitoring")
                    current_risk_level_int = max(current_risk_level_int, 1)

                # Determine final risk string based on the highest risk identified
                if current_risk_level_int == 2:
                    risk = "High"
                elif current_risk_level_int == 1:
                    risk = "Medium"
                else:
                    risk = "Low"

                # Advisory sessions recommendation (conditional on risk being High or Medium)
                # Note: ml_sessionsAttended might be 0 due to imputation if originally missing.
                # The rule should consider if it was *provided* or if it was *imputed to 0 and below 2*.
                # Using sessionsAttended_is_na to distinguish. If it was provided and < 2.
                if not sessionsAttended_is_na and ml_sessionsAttended < 2:
                    if risk == "High" or risk == "Medium":
                        actions.add("Schedule advising session if risk is high or medium if low not necessary")
                elif sessionsAttended_is_na and (risk == "High" or risk == "Medium"):
                    # If sessionsAttended was not provided, but risk is High/Medium, still recommend scheduling.
                    actions.add("Schedule advising session if risk is high or medium if low not necessary")


                # Add general recommendations based on the final determined risk level from ONTOLOGY_RECOMMENDATIONS
                general_recs = ONTOLOGY_RECOMMENDATIONS[risk].split('<br>')
                for rec in general_recs:
                    actions.add(rec.strip())

                # Convert actions set to list for joining
                actions_list = list(actions)

                # Default action if no specific actions were triggered and 'Low' risk general rec wasn't already added (should not happen if ONTOLOGY_RECOMMENDATIONS is used)
                if not actions_list:
                    actions_list = ["No action needed – student performing well"]

                color = "high" if risk == "High" else "medium" if risk == "Medium" else "low"
                color_text = "red" if risk == "High" else "orange" if risk == "Medium" else "green"

                result = {
                    'id': sid,
                    'risk': risk,
                    'color': color,
                    'color_text': color_text,
                    'actions': '<br>'.join(actions_list),
                    'ml_score': f"{ml_score:.3f}"
                }
                print(f"POST request result: {result}")
            except Exception as e: # Catch errors specifically during POST request processing
                import traceback
                print("An error occurred during POST request processing:")
                traceback.print_exc()
                result = {'id': 'Error', 'risk': 'Invalid input', 'color': 'low', 'color_text': 'black', 'actions': str(e), 'ml_score': 'N/A'}

    except Exception as e:
        # Catch any error during GET request or initial processing outside POST block
        import traceback
        print("An unhandled error occurred in the home function (GET request or general processing):")
        traceback.print_exc() # This will print the full traceback to Colab's output
        result = {'id': 'Global Error', 'risk': 'Server Issue', 'color': 'high', 'color_text': 'red', 'actions': f"An unexpected error occurred: {e}", 'ml_score': 'N/A'}

    print("--- Exiting home() function ---")
    return render_template_string(TEMPLATE, ontology_students=ontology_students, result=result)

# Cell 6: Run Flask
# Changed port to 5011 to avoid 'Address already in use' error
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5000,'use_reloader': False}).start()

from google.colab import output
# Changed port to 5011 to avoid 'Address already in use' error
output.serve_kernel_port_as_window(5000)

Upload AUN_SRO.owl


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


Saving AUN_SRO.owx to AUN_SRO (3).owx

Upload best_rf_model.pkl


Saving retrained_best_model.pkl to retrained_best_model (3).pkl
Ontology loaded!
ML model loaded!
Try `serve_kernel_port_as_iframe` instead. 
 * Serving Flask app '__main__'
 * Debug mode: on


<IPython.core.display.Javascript object>

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
